In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import re
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

from sklearn.metrics.pairwise import cosine_similarity
import networkx as netx

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [11]:
newsarticles = pd.read_csv('news_articles.csv')
newsarticles.shape

(180, 6)

In [12]:
newsarticles.head()

,Unnamed: 0,Title,date,Article,Category,PageLinks
0,0,"For some, the pandemic is an opportunity to st...","September 6, 2020","By six every morning, Kinchu Lhamo Bhutia is o...",National,https://kathmandupost.com/national/2020/09/06/...
1,1,Smugglers in Dadeldhura haphazardly fell trees...,"September 6, 2020",Hundreds of trees have been felled down on the...,National,https://kathmandupost.com/sudurpaschim-provinc...
2,2,Around 40 percent government staff haven’t rec...,"September 6, 2020",The Rautahat District Treasury Comptroller’s O...,National,https://kathmandupost.com/national/2020/09/06/...
3,3,Relief distribution programme in Birgunj fails...,"September 6, 2020","On Thursday, many residents of Ranighat in Bir...",National,https://kathmandupost.com/province-no-2/2020/0...
4,4,Job schemes come into question as virus-induce...,"September 6, 2020","Last week, when a group of Nepalis were crossi...",National,https://kathmandupost.com/national/2020/09/06/...


In [13]:
newsarticles.drop('Unnamed: 0',axis=1, inplace=True)

In [23]:
# Category of news
newsarticles.Category.unique()

array(['National', 'Politics', 'Valley', 'Opinion', 'Money', 'Sports',
       'Health', 'Food', 'Science & Technology'], dtype=object)

In [14]:
data = newsarticles.copy()
data.head()

,Title,date,Article,Category,PageLinks
0,"For some, the pandemic is an opportunity to st...","September 6, 2020","By six every morning, Kinchu Lhamo Bhutia is o...",National,https://kathmandupost.com/national/2020/09/06/...
1,Smugglers in Dadeldhura haphazardly fell trees...,"September 6, 2020",Hundreds of trees have been felled down on the...,National,https://kathmandupost.com/sudurpaschim-provinc...
2,Around 40 percent government staff haven’t rec...,"September 6, 2020",The Rautahat District Treasury Comptroller’s O...,National,https://kathmandupost.com/national/2020/09/06/...
3,Relief distribution programme in Birgunj fails...,"September 6, 2020","On Thursday, many residents of Ranighat in Bir...",National,https://kathmandupost.com/province-no-2/2020/0...
4,Job schemes come into question as virus-induce...,"September 6, 2020","Last week, when a group of Nepalis were crossi...",National,https://kathmandupost.com/national/2020/09/06/...


We will be using the pre-trained **Wikipedia 2014 + Gigaword 5** GloVe vectors for word_embedding. Glove word embeddings are the vector representation of the words.

In [7]:
word_embeddings = {}
f = open('glove.6B.100d.txt', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    word_embeddings[word] = coefs
f.close()

In [8]:
len(word_embeddings)

400000

In [15]:
# function to remove stopwords
def remove_stopwords(sen):
    sen_new = " ".join([i for i in sen.split() if i not in stop_words])
    return sen_new

# this function is used to preprocess newsarticles
def preprocess(text):
    sentences = sent_tokenize(text)
    neat_sentences = pd.Series(sentences).str.replace("[^a-zA-Z]", " ")  # removes all special characters
    neat_sentences = [s.lower() for s in neat_sentences]
    neat_sentences = [remove_stopwords(sent) for sent in neat_sentences]
    return neat_sentences,sentences

# vectorizing each sentences in the article
def vectorize_sentences(cleaned_sentences):
    sent_vectors = []
    for i in cleaned_sentences:
        if len(i) != 0:
            vector = sum([word_embeddings.get(w, np.zeros((100,))) for w in i.split()])/(len(i.split())+0.001)
        else:
            vector = np.zeros((100,))
        sent_vectors.append(vector)
    
    return sent_vectors
        


In [16]:
from sklearn.metrics.pairwise import cosine_similarity
def similarity_matrix(sentences,sentence_vectors):
    simi_mat = np.zeros([len(sentences), len(sentences)])
    for i in range(len(sentences)):
        for j in range(len(sentences)):
            if i != j:
                simi_mat[i][j] = cosine_similarity(sentence_vectors[i].reshape(1,100), sentence_vectors[j].reshape(1,100))[0,0]
    return simi_mat

## applying pagerank algorithm 
def pageRank(sentences,simi_matrix):
    nx_graph = netx.from_numpy_array(simi_matrix)
    scores = netx.pagerank(nx_graph)
    return scores
    

In [17]:
def summarize(article_title):
    articletext = newsarticles[newsarticles.Title == article_title].Article
    list_corpus = list(articletext)
    preprocessed_sent, sentences = preprocess(list_corpus[0])
    
    sentence_vector = vectorize_sentences(preprocessed_sent)
    
    sim_matrix = similarity_matrix(preprocessed_sent,sentence_vector)
    
    scores = pageRank(sentences,sim_matrix)
    ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)
    top_sentences =[]
    for i in range(7):
        top_sentences.append(ranked_sentences[i][1])
        
    summary = '.'.join(top_sentences)
    return summary  

In [18]:
def related_articles(search_word):
    df_table = pd.DataFrame(columns = ["Date","Title","ArticleSummary"])
    summaries = []
    articles_df = newsarticles[newsarticles['Category'] == search_word]
    df_table.Date = list(articles_df.date)
    df_table.Title = list(articles_df.Title)
    # create summaries
    for title in list(df_table.Title):
        summaries.append(summarize(title))
    df_table.ArticleSummary = summaries
    #df_table.drop("word_ctr", axis=1, inplace = True)
    
    return df_table

pd.options.display.max_colwidth = 1500

In [19]:
## summary of 'National' newscategory
df_national = related_articles('National')
df_national

,Date,Title,ArticleSummary
0,"September 6, 2020","For some, the pandemic is an opportunity to start new businesses","“But ever since I started helping my father with his business, I got too busy to spend much time on baking.” But that changed during the lockdown..“Friends who had tried my food have always told me to start my own food business but I never really thought I would actually start it in the midst of a pandemic,” said Bhutia..She had always wanted to start a business focusing on serving the kind of Sikkimese food that she grew up eating..“During the lockdown, our family business had to be shut, and I had ample free time to bake, and I resumed baking and shared my baked products with family, friends, and neighbours,” said Manandhar..For Tamang and Bhutia, it was when their friends became repeat customers that they knew their business was going to do well..But when the orders started increasing, I realised having home delivery services makes things much more convenient for customers,” said Manandhar..But between helping her husband’s pashmina business in Kathmandu and raising her children, she had very little time to do anything else."
1,"September 6, 2020",Smugglers in Dadeldhura haphazardly fell trees on pretext of constructing road,"“We had informed the Sub Division Office about the haphazard tree felling but the forest employees did not pay any heed.” According to the Division Forest Office in Dadeldhura, forest officials also found sal logs hidden inside the houses of consumers four days ago..“The spokesperson at the Division Forest Office in Dadeldhura will provide additional information about the tree felling in Rajani Community Forest and its vicinity.” Meanwhile, Bishnu Acharya, chief at the Dadeldhura Forest Office, said, “We found that a large number of trees were felled down in collusion with forest employees in Aalital..“Smugglers, in collusion with forest officials and local political leaders, have been smuggling timber from the Aalital area for a long time..Forest officials are collecting details on the incident.” According to Acharya, two individuals, including the chairman of the Rajani Community Forest, have been arrested to make inquiries about the incident on August 24..The incident came to light on August 22 when forest officials confiscated a large quantity of green sal logs on the road section that connects Godam bazaar in Aalital to Dola Village..Locals said they cannot openly talk about the smugglers and timber smuggling in the forest because of an incident in which an individual, who provided information to the media about timber smuggling, was burnt alive by smugglers a decade ago..However, Santosh Bhatta, chief at the Aalital Sub Division Forest Office, claims he does not k..."
2,"September 6, 2020",Around 40 percent government staff haven’t received their salaries yet,"“As provincial government offices have not submitted their financial details for the 2019-20 fiscal, which ended two months ago, and the payment orders seeking funds for the salaries, not a single penny has been released as salary as of Friday,” said Ishwori Prasad Dhakal, chief of District Treasury Comptroller Office, Rautahat..Around 40 percent of government employees at both the federal and provincial levels across the country haven’t received the salary for Shrawan, according to the Financial Comptroller General’s Office..Usually, the district treasury offices start releasing funds for salaries weeks before the end of the month based on financial details submitted by the concerned government offices..“The district treasury offices have released funds for the salaries of only 60 percent of government employees so far,” said Deputy Financial Comptroller General Bhesh Prasad Bhurtel..For staffers at a government office to receive salaries, the office chief and chief accountant need to sign a payment order requesting the district treasury office to release funds..Even Kathmandu-based district treasury offices have not release

In [21]:
df_national['ArticleSummary'][0]

'“But ever since I started helping my father with his business, I got too busy to spend much time on baking.”  But that changed during the lockdown..“Friends who had tried my food have always told me to start my own food business but I never really thought I would actually start it in the midst of a pandemic,” said Bhutia..She had always wanted to start a business focusing on serving the kind of Sikkimese food that she grew up eating..“During the lockdown, our family business had to be shut, and I had ample free time to bake, and I resumed baking and shared my baked products with family, friends, and neighbours,” said Manandhar..For Tamang and Bhutia, it was when their friends became repeat customers that they knew their business was going to do well..But when the orders started increasing, I realised having home delivery services makes things much more convenient for customers,” said Manandhar..But between helping her husband’s pashmina business in Kathmandu and raising her children, 